In [18]:
'''
Module: analysis_result.ipynb
Author: Kang Hyun Woo
Last Modified: 2025-09-08 09:29
Description: 추산된 실험 결과를 분석하는 노트북 스크립트
'''

'\nModule: analysis_result.ipynb\nAuthor: Kang Hyun Woo\nLast Modified: 2025-09-08 09:29\nDescription: 추산된 실험 결과를 분석하는 노트북 스크립트\n'

In [19]:
import os
import pandas as pd

In [20]:
pd.set_option('display.width', 1000)

# 최대 출력 행 수 설정 (None이면 모든 행 출력)
pd.set_option('display.max_rows', None)

# 최대 출력 열 수 설정 (None이면 모든 열 출력)
pd.set_option('display.max_columns', None)

# 열 내용의 최대 너비 설정
pd.set_option('display.max_colwidth', None)

In [21]:
target_path = 'aggregated_results.csv'
df = pd.read_csv(target_path)

In [22]:
# 중복 확인
# df에 architecture, dataset, encoder, horizon, n_cluster, d_model, beta, seed, postfix가 전부 같은 행이 있는지 확인

# 중복 확인할 컬럼들 정의
duplicate_columns = ['architecture', 'dataset', 'encoder', 'horizon', 
                    'n_cluster', 'd_model', 'beta', 'seed', 'postfix']

duplicate_groups = df.groupby(duplicate_columns).filter(lambda x: len(x) > 1)
#print("\n중복된 그룹들:")
#print(duplicate_groups.sort_values(duplicate_columns))

# 중복된 두 개 이상의 행에 대해 'rse'가 최소인 행을 선택
min_rse_rows = duplicate_groups.loc[duplicate_groups.groupby(duplicate_columns)['rse'].idxmin()]

# df에 중복된 행을 제거하고, 'rse'가 최소인 행만 남김
df_unique = df.drop_duplicates(subset=duplicate_columns, keep=False)
df_unique = pd.concat([df_unique, min_rse_rows]).sort_values(duplicate_columns)

# RSE가 3.0보다 큰 행은 제거
df_unique = df_unique[df_unique['rse'] <= 4.0] 

# 중복 제거 전, 후 행 수 출력하여 비교
print(f"\n중복 제거 전 행 수: {len(df)}")
print(f"중복 제거 후 행 수: {len(df_unique)}")


중복 제거 전 행 수: 4141
중복 제거 후 행 수: 3904


In [23]:
# architecture, dataset, encoder, horizon, n_cluster, d_model, beta가 같고, seed가 다른 행들을 그룹화, postfix 열은 아무거나 상관없음
# 그룹명은 a=<architecture>_d=<dataset>_e=<encoder>_h=<horizon>_n=<n_cluster>_m=<d_model>_b=<beta>_p=<postfix>로 설정
grouped_df = df_unique.groupby(
    ['architecture', 'dataset', 'encoder', 'horizon', 'n_cluster', 'd_model', 'beta', 'postfix']
)

In [24]:
# 그룹 수 출력
print(f"\n총 그룹 수: {len(grouped_df)}")


총 그룹 수: 1253


In [25]:
def print_with_query(grouped_df, query: dict, to_txt=False, show_min=False):
    import io
    import sys
    from datetime import datetime
    
    # 출력을 캡처하기 위한 StringIO 객체
    if to_txt:
        output_buffer = io.StringIO()
        original_stdout = sys.stdout
        sys.stdout = output_buffer
    
    print(f"\nQuery: {query}")
    print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 80)
    
    # grouped_df에서 query에 해당하는 값을 포함한 그룹을 모두 찾아 출력
    matching_groups = 0
    
    for name, group in grouped_df:
        # name은 튜플 형태: (architecture, dataset, encoder, horizon, n_cluster, d_model, beta, postfix)
        # 각 쿼리 조건을 확인
        match = True
        
        # 그룹 튜플의 인덱스와 쿼리 키 매핑
        group_keys = ['architecture', 'dataset', 'encoder', 'horizon', 'n_cluster', 'd_model', 'beta', 'postfix']
        
        for key, query_value in query.items():
            # 'all'인 경우 필터링에서 제외
            if query_value == 'all':
                continue
                
            # 키가 그룹 키에 있는지 확인
            if key in group_keys:
                key_index = group_keys.index(key)
                group_value = name[key_index]
                
                # 값 비교 (문자열과 숫자 모두 고려)
                if str(group_value) != str(query_value):
                    match = False
                    break
        
        # 조건에 맞는 그룹만 출력
        if match:
            matching_groups += 1
            print(f"\nGroup {matching_groups}: {name}")
            print(group.to_string(index=False))
            print(f"Avg. RSE: {group['rse'].mean():.3f}")
            print(f"Std. RSE: {group['rse'].std():.3f}")
            # show_min 옵션이 True인 경우, RSE가 최소인 값을 출력
            if show_min:
                min_rse_row = group.loc[group['rse'].idxmin()]
                print(f"Min RSE: {min_rse_row['rse']:.3f}")
            print('-' * 50)
    
    print(f"\n총 {matching_groups}개의 그룹이 쿼리 조건과 일치합니다.")
    print("=" * 80)
    
    # 파일로 출력하기
    if to_txt:
        # stdout 원복
        sys.stdout = original_stdout
        
        # 출력 내용 가져오기
        output_content = output_buffer.getvalue()
        output_buffer.close()
        
        # 파일에 저장
        with open('analysis_result.txt', 'w', encoding='utf-8') as f:
            f.write(output_content)
        
        print(f"✅ 분석 결과가 'analysis_result.txt' 파일에 저장되었습니다.")
        print(f"📊 총 {matching_groups}개의 그룹이 쿼리 조건과 일치합니다.")
    
    return matching_groups

In [38]:
query = {
    'architecture': 'gru',
    'dataset': 'etth2',
    #'encoder': 'delta',
    #'horizon': 6,
    #'n_cluster': 'unknown',
    #'d_model': '128',
    #'postfix': 'unknown',  # 예시로 postfix를 추가
}

# 사용 예시
print_with_query(grouped_df, query, to_txt=True, show_min=True)

✅ 분석 결과가 'analysis_result.txt' 파일에 저장되었습니다.
📊 총 4개의 그룹이 쿼리 조건과 일치합니다.


4